# Multi-Label
## embedding+专家网络（mmoe，3专家）+4label输出
## 最终目标是输出xxx在不同item下的打分，和多标签一致
## 不同于其他解决方案，考虑了标签之间的关系，且不同标签的打分可比
## 不考虑xxx\xxx\xxx等入口，对进入到xxx的xxx预测对应item的cvr（从xxx到item信息填写页可看作一体）

In [4]:
import datetime
import numpy as np
import x2ndas as pd
import joblib
import warnings
import logging
import os
import gc
import matplotlib.pyplot as plt
import matplotlib.x2tches as mx2tches
import seaborn as sns
import collections
import re
import copy
import tensorflow as tf

import utils_

from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from joblib import x2rallel, delayed
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.inspection import permutation_importance
# from deepctr.feature_column import Sx2rseFeat, DenseFeat
# from deepctr.models import MMOE
from tensorflow import keras

from utils.feature_column import Sx2rseFeat, DenseFeat
from model.mtl import MTL

# pd.set_option('display.max_columns', None)
# pd.set_option('max_row', 500)
warnings.filterwarnings('ignore')
tqdm.x2ndas(desc='x2ndas bar')

# 数据处理

## 数据

In [3]:
# x
df_20220410_20220424 = utils_.load_pickle('../data/df_all_items_u1s_20220410_20220424.pickle')
print(df_20220410_20220424.shape)

df_20220501_20220515 = utils_.load_pickle('../data/df_all_items_u1s_20220501_20220515.pickle')
print(df_20220501_20220515.shape)

df_20220522_20220529 = utils_.load_pickle('../data/df_all_items_u1s_select_ht_20220522_20220529.pickle')
print(df_20220522_20220529.shape)

In [ ]:
# x city
df_20220410_20220424_city = utils_.load_pickle('../data/df_all_items_u1s_city_20220410_20220424.pickle')
print(df_20220410_20220424_city.shape)

df_20220501_20220515_city = utils_.load_pickle('../data/df_all_items_u1s_city_20220501_20220515.pickle')
print(df_20220501_20220515_city.shape)

df_20220522_20220529_city = utils_.load_pickle('../data/df_all_items_u1s_city_select_ht_20220522_20220529.pickle')
print(df_20220522_20220529_city.shape)

In [ ]:
df_20220410_20220424_city['dt'] = df_20220410_20220424_city['dt'].astype(str)
df_20220410_20220424_city['obs_dt'] = df_20220410_20220424_city['dt'].apply(lambda x: '2022-04-24' if x>'2022-04-24' 
                                                                            else '2022-04-17' if x>'2022-04-17' 
                                                                            else '2022-04-10')
print(df_20220410_20220424_city.shape)
df_20220410_20220424_city.head()

In [ ]:
# x added1
df_20220410_20220424_added1 = utils_.load_pickle('../data/df_all_items_u1s_added1_20220410_20220424.pickle')
print(df_20220410_20220424_added1.shape)

df_20220501_20220515_added1 = utils_.load_pickle('../data/df_all_items_u1s_added1_20220501_20220515.pickle')
print(df_20220501_20220515_added1.shape)

df_20220522_20220529_added1 = utils_.load_pickle('../data/df_all_items_u1s_added1_select_ht_20220522_20220529.pickle')
print(df_20220522_20220529_added1.shape)

In [ ]:
list_feats_id_y = ['uid', 
                   'obs_dt', 'dt', 
                   'item', 
                   'label_product_apply_click', 'is_item_info_exposure', 'label_item_info_submit_click', 'label_item_x2ss']

list_feats_x = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_20220410_20220424_combine.pickle')
list_feats_x_city = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_city_20220410_20220424_combine.pickle')
list_feats_x_added1 = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_added1_20220410_20220424_combine.pickle')

print(len(list_feats_x))
print(len(list_feats_x_city))
print(len(list_feats_x_added1))

In [ ]:
# x
df = pd.concat([df_20220410_20220424[list_feats_id_y+list_feats_x], 
                df_20220501_20220515[list_feats_id_y+list_feats_x], 
                df_20220522_20220529[list_feats_id_y+list_feats_x]], axis=0)

df.reset_index(drop=True, inplace=True)
print(df.shape)

# x city
df_city = pd.concat([df_20220410_20220424_city[list_feats_id_y+list_feats_x_city], 
                     df_20220501_20220515_city[list_feats_id_y+list_feats_x_city], 
                     df_20220522_20220529_city[list_feats_id_y+list_feats_x_city]], axis=0)

df_city.reset_index(drop=True, inplace=True)
print(df_city.shape)

# x added1
df_added1 = pd.concat([df_20220410_20220424_added1[list_feats_id_y+list_feats_x_added1], 
                       df_20220501_20220515_added1[list_feats_id_y+list_feats_x_added1], 
                       df_20220522_20220529_added1[list_feats_id_y+list_feats_x_added1]], axis=0)

df_added1.reset_index(drop=True, inplace=True)
print(df_added1.shape)

In [ ]:
df['obs_dt'] = pd.to_datetime(df['obs_dt'])
df['dt'] = pd.to_datetime(df['dt'])

df_city['obs_dt'] = pd.to_datetime(df_city['obs_dt'])
df_city['dt'] = pd.to_datetime(df_city['dt'])

df_added1['obs_dt'] = pd.to_datetime(df_added1['obs_dt'])
df_added1['dt'] = pd.to_datetime(df_added1['dt'])

In [ ]:
%%time
df_all = df.merge(df_city, on=list_feats_id_y, how='left').\
    merge(df_added1, on=list_feats_id_y, how='left')
print(df_all.shape)
df_all.head()

In [ ]:
df_all.groupby(by=['obs_dt', 'dt'])['uid'].count()

In [ ]:
utils_.save_pickle(df_all, '../data/df_select_origin_20220410_20220529.pickle')

## 多标签处理

In [ ]:
df = utils_.load_pickle('../data/df_select_origin_20220410_20220529.pickle')
print(df.shape)
df.head()

In [ ]:
df_count = df[['uid', 'obs_dt', 'dt', 'item']].groupby(by=['uid', 'obs_dt', 'dt'], as_index=False).count()
print(df_count.shape)
df_count['item'].value_counts()

In [ ]:
df['item_id'] = df['item'].apply(lambda x: 0 if x=='item_1' else 1 if x=='item_2' else 2 if x=='item_3' else 3)
df[['item', 'item_id', 'uid']].groupby(['item', 'item_id']).count()

In [ ]:
list_feats_ohe = ['item_id']
list_df_ohe = []
try:
    with tqdm(list_feats_ohe) as t:
        for feat in t:
            df_ohe_feat = utils_.one_hot_encoder(df, feat)
            list_df_ohe.append(df_ohe_feat)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_ohe = pd.concat(list_df_ohe, axis=1)
print(df_ohe.shape)
df_ohe.head()

In [ ]:
df = pd.concat([df, df_ohe], axis=1)
print(df.shape)
df.head()

In [ ]:
df['label_item_info_submit_click_x1'] = df['label_item_info_submit_click'] * df['item_id_0']
df['label_item_info_submit_click_x2'] = df['label_item_info_submit_click'] * df['item_id_1']
df['label_item_info_submit_click_x3'] = df['label_item_info_submit_click'] * df['item_id_2']
df['label_item_info_submit_click_other'] = df['label_item_info_submit_click'] * df['item_id_3']

print(df.shape)
df.head()

In [ ]:
list_feats_x = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_20220410_20220424_combine.pickle')
list_feats_x_city = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_city_20220410_20220424_combine.pickle')
list_feats_x_added1 = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_added1_20220410_20220424_combine.pickle')

print(len(list_feats_x))
print(len(list_feats_x_city))
print(len(list_feats_x_added1))

list_feats = list_feats_x + list_feats_x_city + list_feats_x_added1
print(len(list_feats))

In [ ]:
df = df[['uid', 'obs_dt', 'dt']+
        ['item_id_0', 'item_id_1', 'item_id_2', 'item_id_3']+
        list_feats+
        ['label_item_info_submit_click_x1', 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x3', 'label_item_info_submit_click_other']]
print(df.shape)
df.head()

In [ ]:
# 单日多item去重处理
df_items = df_count[df_count['item']>=2][['uid', 'obs_dt', 'dt']].merge(df, on=['uid', 'obs_dt', 'dt'], how='left')
print(df_items.shape)
df_items.head()

In [ ]:
def func_combine(df_group):
    df_res = df_group.head(1).copy()
    if df_group.shape[0] > 1:
        df_res['item_id_0'] = df_group['item_id_0'].max()
        df_res['item_id_1'] = df_group['item_id_1'].max()
        df_res['item_id_2'] = df_group['item_id_2'].max()
        df_res['item_id_3'] = df_group['item_id_3'].max()
        
        df_res['label_item_info_submit_click_x1'] = df_group['label_item_info_submit_click_x1'].max()
        df_res['label_item_info_submit_click_x2'] = df_group['label_item_info_submit_click_x2'].max()
        df_res['label_item_info_submit_click_x3'] = df_group['label_item_info_submit_click_x3'].max()
        df_res['label_item_info_submit_click_other'] = df_group['label_item_info_submit_click_other'].max()
        
    return df_res


def apply_x2rallel(df_grouped, func):
    try:
        with tqdm(df_grouped) as t:
            list_x2raller = x2rallel(n_jobs=16)(delayed(func)(df_group) for name, df_group in t)
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()
    
    return list_x2raller

In [ ]:
# 合并后去重
df_items_grouped = df_items.groupby(by=['uid', 'obs_dt', 'dt'])
list_df_items_combine = apply_x2rallel(df_items_grouped, func_combine)
df_items_combine = pd.concat(list_df_items_combine, axis=0)
df_items_combine.reset_index(drop=True, inplace=True)
print(df_items_combine.shape)
df_items_combine.head()

In [ ]:
%%time
# 单日单item直接处理
df_item = df_count[df_count['item']==1][['uid', 'obs_dt', 'dt']].merge(df, on=['uid', 'obs_dt', 'dt'], how='left')
print(df_item.shape)
df_item.head()

In [ ]:
df_mtl = pd.concat([df_item, df_items_combine], axis=0)
df_mtl.reset_index(drop=True, inplace=True)
print(df_mtl.shape)
df_mtl.head()

In [ ]:
utils_.save_pickle(df_mtl, '../data/mtl/df_20220410_20220529.pickle')

## 划分训练（包含验证）、测试集

In [ ]:
df = utils_.load_pickle('../data/mtl/df_20220410_20220529.pickle')
print(df.shape)
df.head()

In [ ]:
print(df[df['label_item_info_submit_click_x1']==1].shape[0], df[df['label_item_info_submit_click_x1']==1].shape[0]/df.shape[0])
print(df[df['label_item_info_submit_click_x2']==1].shape[0], df[df['label_item_info_submit_click_x2']==1].shape[0]/df.shape[0])
print(df[df['label_item_info_submit_click_x3']==1].shape[0], df[df['label_item_info_submit_click_x3']==1].shape[0]/df.shape[0])
print(df[df['label_item_info_submit_click_other']==1].shape[0], df[df['label_item_info_submit_click_other']==1].shape[0]/df.shape[0])

In [ ]:
list_feats_x = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_20220410_20220424_combine.pickle')
list_feats_x_city = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_city_20220410_20220424_combine.pickle')
list_feats_x_added1 = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_added1_20220410_20220424_combine.pickle')

print(len(list_feats_x))
print(len(list_feats_x_city))
print(len(list_feats_x_added1))

df_id = df[['uid', 'obs_dt', 'dt', 'item_id_0', 'item_id_1', 'item_id_2', 'item_id_3']]
df_y = df[['label_item_info_submit_click_x1', 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x3', 'label_item_info_submit_click_other']]
df_X = df[list_feats_x+list_feats_x_city+list_feats_x_added1]

print(df_id.shape)
print(df_y.shape)
print(df_X.shape)

In [ ]:
# 划分训练集、测试集，8:2
df_X_train, df_X_test, df_y_train, df_y_test, df_id_train, df_id_test = \
    train_test_split(df_X, df_y, df_id, test_size=0.2, random_state=2022)
print(df_X_train.shape)
print(df_X_test.shape)
print(df_y_train.shape)
print(df_y_test.shape)
print(df_id_train.shape)
print(df_id_test.shape)

In [ ]:
utils_.save_pickle(df_id_train, '../data/mtl/df_id_train_20220410_20220529.pickle')
utils_.save_pickle(df_id_test, '../data/mtl/df_id_test_20220410_20220529.pickle')
utils_.save_pickle(df_y_train, '../data/mtl/df_y_train_20220410_20220529.pickle')
utils_.save_pickle(df_y_test, '../data/mtl/df_y_test_20220410_20220529.pickle')
utils_.save_pickle(df_X_train, '../data/mtl/df_X_train_20220410_20220529.pickle')
utils_.save_pickle(df_X_test, '../data/mtl/df_X_test_20220410_20220529.pickle')

# 入参处理

In [ ]:
df_id_train = utils_.load_pickle('../data/mtl/df_id_train_20220410_20220529.pickle')
df_y_train = utils_.load_pickle('../data/mtl/df_y_train_20220410_20220529.pickle')
df_X_train = utils_.load_pickle('../data/mtl/df_X_train_20220410_20220529.pickle')

print(df_id_train.shape)
print(df_y_train.shape)
print(df_X_train.shape)

In [ ]:
list_feats_x = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_20220410_20220424_combine.pickle')
list_feats_x_city = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_city_20220410_20220424_combine.pickle')
list_feats_x_added1 = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_added1_20220410_20220424_combine.pickle')

print(len(list_feats_x))
print(len(list_feats_x_city))
print(len(list_feats_x_added1))

list_feats = list_feats_x + list_feats_x_city + list_feats_x_added1
print(len(list_feats))

In [ ]:
sx2rse_features = pd.read_csv('../data/mtl/sx2rse_features.txt', sep='\t', encoding='utf-8')
print(sx2rse_features.shape)
sx2rse_features.head()

In [ ]:
list_feats_sx2rse = list(sx2rse_features['feats'])
print(len(list_feats_sx2rse))
list_feats_sx2rse[:10]

In [ ]:
utils_.save_pickle(list_feats_sx2rse, '../data/mtl/list_sx2rse_features.pickle')

In [ ]:
df_X_train[list_feats_sx2rse].head()

In [ ]:
list_dense_features = [x for x in list_feats if x not in list_feats_sx2rse]
print(len(list_dense_features))
list_dense_features[:10]

In [ ]:
utils_.save_pickle(list_dense_features, '../data/mtl/list_dense_features.pickle')

In [ ]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(df_X_train[list_dense_features])
joblib.dump(ss, '../data/mtl/ss.pickle')

In [ ]:
df_X_train_dense = pd.DataFrame(X_train_ss, columns=list_dense_features)
print(df_X_train_dense.shape)
df_X_train_dense.head()

In [ ]:
utils_.save_pickle(df_X_train_dense, '../data/mtl/df_X_train_20220410_20220529_dense_ss.pickle')

In [ ]:
dict_lbe = {}
list_df_X_train_sx2rse = []

try:
    with tqdm(list_feats_sx2rse) as t:
        for x in t:
            lbe = LabelEncoder()
            df_X_train_sx2rse_each = pd.DataFrame(lbe.fit_transform(df_X_train[x]), columns=[x])
            dict_lbe[x] = lbe
            list_df_X_train_sx2rse.append(df_X_train_sx2rse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

joblib.dump(dict_lbe, '../data/mtl/dict_lbe.pickle')
df_X_train_sx2rse = pd.concat(list_df_X_train_sx2rse, axis=1)
print(df_X_train_sx2rse.shape)
df_X_train_sx2rse.head()

In [ ]:
utils_.save_pickle(df_X_train_sx2rse, '../data/mtl/df_X_train_20220410_20220529_sx2rse_lbe.pickle')

In [ ]:
df_X_train_transform = pd.concat([df_X_train_sx2rse, df_X_train_dense], axis=1)
print(df_X_train_transform.shape)
df_X_train_transform.head()

In [ ]:
utils_.save_pickle(df_X_train_transform, '../data/mtl/df_X_train_20220410_20220529_transform.pickle')

# 模型

* 结构

In [ ]:
df_X_train_transform = utils_.load_pickle('../data/mtl/df_X_train_20220410_20220529_transform.pickle')
print(df_X_train_transform.shape)
df_X_train_transform.head()

In [ ]:
list_sx2rse_features = utils_.load_pickle('../data/mtl/list_sx2rse_features.pickle')
list_dense_features = utils_.load_pickle('../data/mtl/list_dense_features.pickle')

print(len(list_sx2rse_features))
print(len(list_dense_features))

In [ ]:
feature_columns = [Sx2rseFeat(x, df_X_train_transform[x].max() + 1, embedding_dim=4) for x in list_sx2rse_features] + \
    [DenseFeat(x, 1, ) for x in list_dense_features]
feature_columns[40:50]

In [ ]:
utils_.save_pickle(feature_columns, '../data/mtl/feature_columns.pickle')

In [ ]:
model = MTL(feature_columns, 
            num_experts=3, expert_dnn_hidden_units=(256, 128, 64), 
            gate_dnn_hidden_units=(), 
            l2_reg_embedding=0.00001, 
            l2_reg_dnn=0, 
            seed=2022, 
            dnn_dropout=0, 
            dnn_activation='relu', 
            dnn_use_bn=False, 
            label_types=('binary', 'binary', 'binary', 'binary'), label_names=('item_1', 'item_2', 'item_3', 'item_s'))

In [ ]:
from tensorflow.keras.losses import Loss


class focal_loss(Loss):
    def __init__(self, alpha=0.25, gamma=2, **kwargs):
        super(focal_loss,self).__init__(**kwargs)
        self.gamma = gamma
        self.alpha = alpha
    
    def call(self, y_true, y_pred):
        zeros = tf.zeros_like(y_pred, dtype=y_pred.dtype)
        pos_p_sub = tf.where(y_true>zeros, y_true-y_pred, zeros)
        neg_p_sub = tf.where(y_true>zeros, zeros, y_pred)
        per_entry_cross_ent = - self.alpha * (pos_p_sub ** self.gamma) * tf.math.log(tf.clip_by_value(y_pred, 1e-8, 1.0)) \
                              - (1-self.alpha) * (neg_p_sub ** self.gamma) * tf.math.log(tf.clip_by_value(1.0-y_pred, 1e-8, 1.0))
        loss = tf.reduce_sum(per_entry_cross_ent)
        return loss

In [ ]:
# model.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
#               loss=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], 
#               metrics=['binary_crossentropy'])

# 损失函数修改
# model.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
#               loss=[focal_loss(), focal_loss(alpha=0.1), focal_loss(alpha=0.1), focal_loss(alpha=0.01)], 
#               metrics=['binary_crossentropy'])

# 损失函数修改 & 各损失权重
model.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
              loss=[focal_loss(), focal_loss(alpha=0.1), focal_loss(alpha=0.1), focal_loss(alpha=0.01)], 
              metrics=['binary_crossentropy'], 
              loss_weights=[1, 10, 10, 100])

In [ ]:
help(model.compile)

In [ ]:
df_y_train = utils_.load_pickle('../data/mtl/df_y_train_20220410_20220529.pickle')
print(df_y_train.shape)
df_y_train.head()

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', min_lr=1e-6), 
    tf.keras.callbacks.ModelCheckpoint(filex2th='../data/mtl/check_point/multi_label_focal_loss_weight/weights-improvement-{epoch:03d}-{val_loss:.4f}.hdf5', 
#     tf.keras.callbacks.ModelCheckpoint(filex2th='../data/mtl/check_point/multi_label_focal_loss/weights-improvement-{epoch:03d}-{val_loss:.4f}.hdf5', 
#     tf.keras.callbacks.ModelCheckpoint(filex2th='../data/mtl/check_point/multi_label/weights-improvement-{epoch:03d}-{val_loss:.4f}.hdf5', 
                                       monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min'), 
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', x2tience=10, restore_best_weights=True)
]

* 训练

In [ ]:
history = model.fit(
    x=df_X_train_transform.values, 
    y=[df_y_train['label_item_info_submit_click_x1'].values, df_y_train['label_item_info_submit_click_x2'].values, df_y_train['label_item_info_submit_click_x3'].values, df_y_train['label_item_info_submit_click_other'].values], 
    batch_size=1024, 
    epochs=1000, 
    verbose=2, 
    callbacks=callbacks, 
    validation_split=0.2)

In [ ]:
# utils_.save_pickle(history.history, '../data/mtl/check_point/multi_label/history_20220623.pickle')
# utils_.save_pickle(history.history, '../data/mtl/check_point/multi_label_focal_loss/history_20220705.pickle')
utils_.save_pickle(history.history, '../data/mtl/check_point/multi_label_focal_loss_weight/history_20220706.pickle')

In [ ]:
# tf.saved_model.save(model, '../data/mtl/model/multi_label/')
# tf.saved_model.save(model, '../data/mtl/model/multi_label_focal_loss/')
tf.saved_model.save(model, '../data/mtl/model/multi_label_focal_loss_weight/')

* 训练效果

In [ ]:
# dict_fit_history = utils_.load_pickle('../data/mtl/check_point/multi_label/history_20220623.pickle')
# dict_fit_history = utils_.load_pickle('../data/mtl/check_point/multi_label_focal_loss/history_20220705.pickle')
dict_fit_history = utils_.load_pickle('../data/mtl/check_point/multi_label_focal_loss_weight/history_20220706.pickle')
df_fit_history = pd.DataFrame(dict_fit_history)
print(df_fit_history.shape)
df_fit_history.head()

In [ ]:
sns.lineplot(data=df_fit_history[['loss', 'val_loss']])

In [ ]:
sns.lineplot(data=df_fit_history[['output_1_loss', 'val_output_1_loss']])

In [ ]:
# 重制网络结构，加载最优权重
model_restore = MTL(feature_columns, 
                    num_experts=3, expert_dnn_hidden_units=(256, 128, 64), 
                    gate_dnn_hidden_units=(), 
                    l2_reg_embedding=0.00001, 
                    l2_reg_dnn=0, 
                    seed=2022, 
                    dnn_dropout=0, 
                    dnn_activation='relu', 
                    dnn_use_bn=False, 
                    label_types=('binary', 'binary', 'binary', 'binary'), label_names=('item_1', 'item_2', 'item_3', 'Other'))

In [ ]:
model_restore.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
#                       loss=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], 
                      loss=[focal_loss(), focal_loss(alpha=0.1), focal_loss(alpha=0.1), focal_loss(alpha=0.01)], 
                      metrics=['binary_crossentropy'], 
                      loss_weights=[1, 10, 10, 100]
                     )
model_restore.build(input_shape=(df_X_train_transform.values.shape[0], df_X_train_transform.values.shape[1]))
# model_restore.load_weights('../data/mtl/check_point/multi_label/weights-improvement-002-0.1225.hdf5')
# model_restore.load_weights('../data/mtl/check_point/multi_label_focal_loss/weights-improvement-003-11.2687.hdf5')
model_restore.load_weights('../data/mtl/check_point/multi_label_focal_loss_weight/weights-improvement-003-19.5802.hdf5')

In [ ]:
pred_train = model_restore.predict(df_X_train_transform.values, batch_size=1024)

In [ ]:
df_y = pd.concat([df_y_train.reset_index(drop=True), 
                  pd.DataFrame(pred_train[0], columns=['label_item_info_submit_click_x1_pred']), 
                  pd.DataFrame(pred_train[1], columns=['label_item_info_submit_click_x2_pred']), 
                  pd.DataFrame(pred_train[2], columns=['label_item_info_submit_click_x3_pred']), 
                  pd.DataFrame(pred_train[3], columns=['label_item_info_submit_click_other_pred'])], 
                 axis=1)
print(df_y.shape)
df_y.head()

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_x1'], df_y['label_item_info_submit_click_x1_pred'])

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_x2'], df_y['label_item_info_submit_click_x2_pred'])

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_x3'], df_y['label_item_info_submit_click_x3_pred'])

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_other'], df_y['label_item_info_submit_click_other_pred'])

* 测试

In [ ]:
# 数据转换
df_id_test = utils_.load_pickle('../data/mtl/df_id_test_20220410_20220529.pickle')
df_y_test = utils_.load_pickle('../data/mtl/df_y_test_20220410_20220529.pickle')
df_X_test = utils_.load_pickle('../data/mtl/df_X_test_20220410_20220529.pickle')

print(df_id_test.shape)
print(df_y_test.shape)
print(df_X_test.shape)

In [ ]:
list_sx2rse_features = utils_.load_pickle('../data/mtl/list_sx2rse_features.pickle')
list_dense_features = utils_.load_pickle('../data/mtl/list_dense_features.pickle')

print(len(list_sx2rse_features))
print(len(list_dense_features))

In [ ]:
ss = joblib.load('../data/mtl/ss.pickle')
X_test_ss = ss.transform(df_X_test[list_dense_features])
print(X_test_ss.shape)

In [ ]:
df_X_test_dense_ss = pd.DataFrame(X_test_ss, columns=list_dense_features)
print(df_X_test_dense_ss.shape)
df_X_test_dense_ss.head()

In [ ]:
dict_lbe = joblib.load('../data/mtl/dict_lbe.pickle')
list_df_X_test_sx2rse = []

try:
    with tqdm(list_sx2rse_features) as t:
        for x in t:
            print(x)
            if x == 'x1':
                df_X_test[x].replace({38:0}, inplace=True)
            if x == 'x2':
                df_X_test[x].replace({339:-1}, inplace=True)
            if x == 'x3':
                for y in [140425, 140923, 141025, 150421, 150428, 152526, 210122, 210124, 230125, 230184, 230605, 231225, 341825, 420529, 450421, 522625, 610622, 610929, 611026, 620522, 621226, 622901, 654023]:
                    df_X_test[x].replace({y:-1}, inplace=True)
            df_X_test_sx2rse_each = pd.DataFrame(dict_lbe[x].transform(df_X_test[x]), columns=[x])
            list_df_X_test_sx2rse.append(df_X_test_sx2rse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_X_test_sx2rse = pd.concat(list_df_X_test_sx2rse, axis=1)
print(df_X_test_sx2rse.shape)
df_X_test_sx2rse.head()

In [ ]:
df_X_test_transform = pd.concat([df_X_test_sx2rse, df_X_test_dense_ss], axis=1)
print(df_X_test_transform.shape)
df_X_test_transform.head()

In [ ]:
utils_.save_pickle(df_X_test_transform, '../data/mtl/df_X_test_transform_20220410_20220529.pickle')

In [ ]:
df_X_test_transform = utils_.load_pickle('../data/mtl/df_X_test_transform_20220410_20220529.pickle')
print(df_X_test_transform.shape)
df_X_test_transform.head()

In [ ]:
# 加载模型
feature_columns = utils_.load_pickle('../data/mtl/feature_columns.pickle')

model_restore = MTL(feature_columns, 
                    num_experts=3, expert_dnn_hidden_units=(256, 128, 64), 
                    gate_dnn_hidden_units=(), 
                    l2_reg_embedding=0.00001, 
                    l2_reg_dnn=0, 
                    seed=2022, 
                    dnn_dropout=0, 
                    dnn_activation='relu', 
                    dnn_use_bn=False, 
                    label_types=('binary', 'binary', 'binary', 'binary'), label_names=('item_1', 'item_2', 'item_3', 'Other'))
model_restore.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
#                       loss=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], 
                      loss=[focal_loss(), focal_loss(alpha=0.1), focal_loss(alpha=0.1), focal_loss(alpha=0.01)], 
                      metrics=['binary_crossentropy'], 
                      loss_weights=[1, 10, 10, 100]
                     )
model_restore.build(input_shape=(df_X_test_transform.values.shape[0], df_X_test_transform.values.shape[1]))
# model_restore.load_weights('../data/mtl/check_point/multi_label/weights-improvement-002-0.1225.hdf5')
# model_restore.load_weights('../data/mtl/check_point/multi_label_focal_loss/weights-improvement-003-11.2687.hdf5')
model_restore.load_weights('../data/mtl/check_point/multi_label_focal_loss_weight/weights-improvement-003-19.5802.hdf5')

In [ ]:
df_y_test = utils_.load_pickle('../data/mtl/df_y_test_20220410_20220529.pickle')
print(df_y_test.shape)

In [ ]:
# 测试数据
pred_test = model_restore.predict(df_X_test_transform.values, batch_size=1024)

df_y = pd.concat([df_y_test.reset_index(drop=True), 
                  pd.DataFrame(pred_test[0], columns=['label_item_info_submit_click_x1_pred']), 
                  pd.DataFrame(pred_test[1], columns=['label_item_info_submit_click_x2_pred']), 
                  pd.DataFrame(pred_test[2], columns=['label_item_info_submit_click_x3_pred']), 
                  pd.DataFrame(pred_test[3], columns=['label_item_info_submit_click_other_pred'])], 
                 axis=1)
print(df_y.shape)
df_y.head()

In [ ]:
# 测试效果
# 整体
# AUC
roc_auc_score(df_y['label_item_info_submit_click_x1'], df_y['label_item_info_submit_click_x1_pred'])

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_x2'], df_y['label_item_info_submit_click_x2_pred'])

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_x3'], df_y['label_item_info_submit_click_x3_pred'])

In [ ]:
roc_auc_score(df_y['label_item_info_submit_click_other'], df_y['label_item_info_submit_click_other_pred'])

In [ ]:
# Recall@30%
def cal_recall(df_y, y_true, y_pred, threshold=0.3):
    df_y_sort = df_y.copy()
    df_y_sort.sort_values(by=y_pred, ascending=False, inplace=True)
    df_y_sort.reset_index(drop=True, inplace=True)
    
    df_y_sort['label_recall'] = 0
    df_y_sort.loc[:int(df_y_sort.shape[0]*threshold), 'label_recall'] = 1
    
    dict_cr = classification_report(df_y_sort[y_true], df_y_sort['label_recall'], output_dict=True)
    
    return dict_cr['1.0']['recall']

In [ ]:
cal_recall(df_y, 'label_item_info_submit_click_x1', 'label_item_info_submit_click_x1_pred')

In [ ]:
cal_recall(df_y, 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x2_pred')

In [ ]:
cal_recall(df_y, 'label_item_info_submit_click_x3', 'label_item_info_submit_click_x3_pred')

In [ ]:
cal_recall(df_y, 'label_item_info_submit_click_other', 'label_item_info_submit_click_other_pred')

In [ ]:
df_id_test = utils_.load_pickle('../data/mtl/df_id_test_20220410_20220529.pickle')
print(df_id_test.shape)

In [ ]:
df_uid_y = pd.concat([df_id_test.reset_index(drop=True), df_y], axis=1)
print(df_uid_y.shape)
df_uid_y.head()

In [ ]:
# 测试效果
# 独立
df_uid_y_x1 = df_uid_y[df_uid_y['item_id_0']==1]
df_uid_y_x2 = df_uid_y[df_uid_y['item_id_1']==1]
df_uid_y_x3 = df_uid_y[df_uid_y['item_id_2']==1]
df_uid_y_other = df_uid_y[df_uid_y['item_id_3']==1]

print(df_uid_y_x1.shape)
print(df_uid_y_x2.shape)
print(df_uid_y_x3.shape)
print(df_uid_y_other.shape)

In [ ]:
# AUC
print('x1_item：{x1}'.format(x1=roc_auc_score(df_uid_y_x1['label_item_info_submit_click_x1'], df_uid_y_x1['label_item_info_submit_click_x1_pred'])))
print('item：{x2}'.format(x2=roc_auc_score(df_uid_y_x2['label_item_info_submit_click_x2'], df_uid_y_x2['label_item_info_submit_click_x2_pred'])))
print('item：{x3}'.format(x3=roc_auc_score(df_uid_y_x3['label_item_info_submit_click_x3'], df_uid_y_x3['label_item_info_submit_click_x3_pred'])))
print('s：{other}'.format(other=roc_auc_score(df_uid_y_other['label_item_info_submit_click_other'], df_uid_y_other['label_item_info_submit_click_other_pred'])))

In [ ]:
# Recall@30%
print('x1_item：{x1}'.format(x1=cal_recall(df_uid_y_x1, 'label_item_info_submit_click_x1', 'label_item_info_submit_click_x1_pred')))
print('x2_item：{x2}'.format(x2=cal_recall(df_uid_y_x2, 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x2_pred')))
print('x3_item：{x3}'.format(x3=cal_recall(df_uid_y_x3, 'label_item_info_submit_click_x3', 'label_item_info_submit_click_x3_pred')))
print('s：{other}'.format(other=cal_recall(df_uid_y_other, 'label_item_info_submit_click_other', 'label_item_info_submit_click_other_pred')))

# 与现有模型比较

* 数据

In [ ]:
# X
df = pd.read_csv('../data/zfbx_aspiration_sample_label_feature_all_items_u1s_select_ht_2022-06-05_7.txt', sep='\t', encoding='utf-8')

print(df.shape)
df.head()

In [ ]:
# X city
df_city = pd.read_csv('../data/zfbx_aspiration_sample_label_feature_city_all_items_u1s_select_ht_20220605.txt', sep='\t', encoding='utf-8')

print(df_city.shape)
df_city.head()

In [ ]:
# X added1
df_added1 = pd.read_csv('../data/zfbx_aspiration_sample_label_feature_added1_all_items_u1s_select_ht_2022-06-05_7.txt', sep='\t', encoding='utf-8')

print(df_added1.shape)
df_added1.head()

In [ ]:
list_feats_id_y = ['uid', 
                   'obs_dt', 'dt', 
                   'item', 
                   'label_product_apply_click', 'is_item_info_exposure', 'label_item_info_submit_click', 'label_item_x2ss']

list_feats_x = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_20220410_20220424_combine.pickle')
list_feats_x_city = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_city_20220410_20220424_combine.pickle')
list_feats_x_added1 = utils_.load_pickle('../data/list_feats/list_feats_x_ht_all_items_u1s_added1_20220410_20220424_combine.pickle')

list_feats = list_feats_x + list_feats_x_city + list_feats_x_added1

print(len(list_feats_x))
print(len(list_feats_x_city))
print(len(list_feats_x_added1))
print(len(list_feats))

In [ ]:
df['obs_dt'] = pd.to_datetime(df['obs_dt'])
df['dt'] = pd.to_datetime(df['dt'])

df_city['obs_dt'] = pd.to_datetime(df_city['obs_dt'])
df_city['dt'] = pd.to_datetime(df_city['dt'])

df_added1['obs_dt'] = pd.to_datetime(df_added1['obs_dt'])
df_added1['dt'] = pd.to_datetime(df_added1['dt'])

In [ ]:
%%time
df_all = df[list_feats_id_y+list_feats_x].merge(df_city[list_feats_id_y+list_feats_x_city], on=list_feats_id_y, how='left').\
    merge(df_added1[list_feats_id_y+list_feats_x_added1], on=list_feats_id_y, how='left')
print(df_all.shape)
df_all.head()

In [ ]:
df_all.groupby(by=['obs_dt', 'dt'])['uid'].count()

In [ ]:
df_count = df_all[['uid', 'obs_dt', 'dt', 'item']].groupby(by=['uid', 'obs_dt', 'dt'], as_index=False).count()
print(df_count.shape)
df_count['item'].value_counts()

In [ ]:
df_all['item_id'] = df_all['item'].apply(lambda x: 0 if x=='x1_item' else 1 if x=='x2_item' else 2 if x=='x3_item' else 3)
df_all[['item', 'item_id', 'uid']].groupby(['item', 'item_id']).count()

In [ ]:
list_feats_ohe = ['item_id']
list_df_ohe = []
try:
    with tqdm(list_feats_ohe) as t:
        for feat in t:
            df_ohe_feat = utils_.one_hot_encoder(df_all, feat)
            list_df_ohe.append(df_ohe_feat)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_ohe = pd.concat(list_df_ohe, axis=1)
print(df_ohe.shape)
df_ohe.head()

In [ ]:
df_all = pd.concat([df_all, df_ohe], axis=1)
print(df_all.shape)
df_all.head()

In [ ]:
df_all['label_item_info_submit_click_x1'] = df_all['label_item_info_submit_click'] * df_all['item_id_0']
df_all['label_item_info_submit_click_x2'] = df_all['label_item_info_submit_click'] * df_all['item_id_1']
df_all['label_item_info_submit_click_x3'] = df_all['label_item_info_submit_click'] * df_all['item_id_2']
df_all['label_item_info_submit_click_other'] = df_all['label_item_info_submit_click'] * df_all['item_id_3']

print(df_all.shape)
df_all.head()

In [ ]:
df_all = df_all[['uid', 'obs_dt', 'dt']+
                ['item_id_0', 'item_id_1', 'item_id_2', 'item_id_3']+
                list_feats+
                ['label_item_info_submit_click_x1', 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x3', 'label_item_info_submit_click_other']]
print(df_all.shape)
df_all.head()

In [ ]:
# 单日多item去重处理
df_items = df_count[df_count['item']>=2][['uid', 'obs_dt', 'dt']].merge(df_all, on=['uid', 'obs_dt', 'dt'], how='left')
print(df_items.shape)
df_items.head()

In [ ]:
# 合并后去重
df_items_grouped = df_items.groupby(by=['uid', 'obs_dt', 'dt'])
list_df_items_combine = apply_x2rallel(df_items_grouped, func_combine)
df_items_combine = pd.concat(list_df_items_combine, axis=0)
df_items_combine.reset_index(drop=True, inplace=True)
print(df_items_combine.shape)
df_items_combine.head()

In [ ]:
%%time
# 单日单item直接处理
df_item = df_count[df_count['item']==1][['uid', 'obs_dt', 'dt']].merge(df_all, on=['uid', 'obs_dt', 'dt'], how='left')
print(df_item.shape)
df_item.head()

In [ ]:
df_mtl = pd.concat([df_item, df_items_combine], axis=0)
df_mtl.reset_index(drop=True, inplace=True)
print(df_mtl.shape)
df_mtl.head()

In [ ]:
df_id = df_mtl[['uid', 'obs_dt', 'dt', 'item_id_0', 'item_id_1', 'item_id_2', 'item_id_3']]
df_y = df_mtl[['label_item_info_submit_click_x1', 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x3', 'label_item_info_submit_click_other']]
df_X = df_mtl[list_feats]

print(df_id.shape)
print(df_y.shape)
print(df_X.shape)

* 测试

In [ ]:
list_sx2rse_features = utils_.load_pickle('../data/mtl/list_sx2rse_features.pickle')
list_dense_features = utils_.load_pickle('../data/mtl/list_dense_features.pickle')

print(len(list_sx2rse_features))
print(len(list_dense_features))

In [ ]:
ss = joblib.load('../data/mtl/ss.pickle')
X_ss = ss.transform(df_X[list_dense_features])
print(X_ss.shape)

In [ ]:
df_X_dense_ss = pd.DataFrame(X_ss, columns=list_dense_features)
print(df_X_dense_ss.shape)
df_X_dense_ss.head()

In [ ]:
dict_lbe = joblib.load('../data/mtl/dict_lbe.pickle')
list_df_X_sx2rse = []

try:
    with tqdm(list_sx2rse_features) as t:
        for x in t:
            print(x)
            if x == 'x1':
                df_X[x].replace({217:216}, inplace=True)
            if x == 'x2':
                df_X[x].replace({38:0}, inplace=True)
            if x == 'x3':
                for y in [213, 217]:
                    df_X[x].replace({y:-1}, inplace=True)
#             if x == 'permit_month':
#                 df_X[x].replace({49:48}, inplace=True)
            if x == 'x4':
                df_X[x].replace({236:0}, inplace=True)
            if x == 'x5':
                df_X[x].replace({213:-1}, inplace=True)
            if x == 'x6':
                for y in [213, 217, 309]:
                    df_X[x].replace({y:0}, inplace=True)
            if x == 'x7':
                df_X[x].replace({217:216}, inplace=True)
            if x == 'x8':
                for y in [0, 217]:
                    df_X[x].replace({y:-1}, inplace=True)
            if x == 'x9':
                for y in [140922, 141034, 220182, 222403, 230227, 230521, 231083, 231102, 341825, 431223, 511129, 513434, 620723, 621124, 630222, 632802, 640521, 654023]:
                    df_X[x].replace({y:-1}, inplace=True)
            if x == 'x10':
                df_X[x].replace({236:-1}, inplace=True)        
            if x == 'x11':
                for y in [236, 289]:
                    df_X[x].replace({y:-1}, inplace=True)
                    
            df_X_sx2rse_each = pd.DataFrame(dict_lbe[x].transform(df_X[x]), columns=[x])
            list_df_X_sx2rse.append(df_X_sx2rse_each)
except KeyboardInterrupt:
    t.close()
    raise
t.close()

df_X_sx2rse = pd.concat(list_df_X_sx2rse, axis=1)
print(df_X_sx2rse.shape)
df_X_sx2rse.head()

In [ ]:
df_X_transform = pd.concat([df_X_sx2rse, df_X_dense_ss], axis=1)
print(df_X_transform.shape)
df_X_transform.head()

In [ ]:
# 加载模型
feature_columns = utils_.load_pickle('../data/mtl/feature_columns.pickle')

model_restore = MTL(feature_columns, 
                    num_experts=3, expert_dnn_hidden_units=(256, 128, 64), 
                    gate_dnn_hidden_units=(), 
                    l2_reg_embedding=0.00001, 
                    l2_reg_dnn=0, 
                    seed=2022, 
                    dnn_dropout=0, 
                    dnn_activation='relu', 
                    dnn_use_bn=False, 
                    label_types=('binary', 'binary', 'binary', 'binary'), label_names=('item_1', 'item_2', 'item_3', 'Other'))
model_restore.compile(optimizer=keras.optimizers.Adam(lr=0.001), 
#                       loss=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], 
                      loss=[focal_loss(), focal_loss(alpha=0.1), focal_loss(alpha=0.1), focal_loss(alpha=0.01)], 
                      metrics=['binary_crossentropy'], 
                      loss_weights=[1, 10, 10, 100]
                     )
model_restore.build(input_shape=(df_X_transform.values.shape[0], df_X_transform.values.shape[1]))
# model_restore.load_weight/s('../data/mtl/check_point/multi_label/weights-improvement-002-0.1225.hdf5')
# model_restore.load_weights('../data/mtl/check_point/multi_label_focal_loss/weights-improvement-003-11.2687.hdf5')
model_restore.load_weights('../data/mtl/check_point/multi_label_focal_loss_weight/weights-improvement-003-19.5802.hdf5')

In [ ]:
# 测试数据
pred_test = model_restore.predict(df_X_transform.values, batch_size=1024)

df_y_pred = pd.concat([df_y.reset_index(drop=True), 
                       pd.DataFrame(pred_test[0], columns=['label_item_info_submit_click_x1_pred']), 
                       pd.DataFrame(pred_test[1], columns=['label_item_info_submit_click_x2_pred']), 
                       pd.DataFrame(pred_test[2], columns=['label_item_info_submit_click_x3_pred']), 
                       pd.DataFrame(pred_test[3], columns=['label_item_info_submit_click_other_pred'])], 
                      axis=1)
print(df_y_pred.shape)
df_y_pred.head()

In [ ]:
# 整体
# AUC
print('x1_item：{x1}'.format(x1=roc_auc_score(df_y_pred['label_item_info_submit_click_x1'], df_y_pred['label_item_info_submit_click_x1_pred'])))
print('x2_item：{x2}'.format(x2=roc_auc_score(df_y_pred['label_item_info_submit_click_x2'], df_y_pred['label_item_info_submit_click_x2_pred'])))
print('x3_item：{x3}'.format(x3=roc_auc_score(df_y_pred['label_item_info_submit_click_x3'], df_y_pred['label_item_info_submit_click_x3_pred'])))
print('s：{other}'.format(other=roc_auc_score(df_y_pred['label_item_info_submit_click_other'], df_y_pred['label_item_info_submit_click_other_pred'])))

In [ ]:
# Recall@30%
print('x1_item：{x1}'.format(x1=cal_recall(df_y_pred, 'label_item_info_submit_click_x1', 'label_item_info_submit_click_x1_pred')))
print('x2_item：{x2}'.format(x2=cal_recall(df_y_pred, 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x2_pred')))
print('x3_item：{x3}'.format(x3=cal_recall(df_y_pred, 'label_item_info_submit_click_x3', 'label_item_info_submit_click_x3_pred')))
print('s：{other}'.format(other=cal_recall(df_y_pred, 'label_item_info_submit_click_other', 'label_item_info_submit_click_other_pred')))

In [ ]:
df_uid_y = pd.concat([df_id.reset_index(drop=True), df_y_pred], axis=1)
print(df_uid_y.shape)
df_uid_y.head()

In [ ]:
# 测试效果
# 独立
df_uid_y_x1 = df_uid_y[df_uid_y['item_id_0']==1]
df_uid_y_x2 = df_uid_y[df_uid_y['item_id_1']==1]
df_uid_y_x3 = df_uid_y[df_uid_y['item_id_2']==1]
df_uid_y_other = df_uid_y[df_uid_y['item_id_3']==1]

print(df_uid_y_x1.shape)
print(df_uid_y_x2.shape)
print(df_uid_y_x3.shape)
print(df_uid_y_other.shape)

In [ ]:
# AUC
print('x1_item：{x1}'.format(x1=roc_auc_score(df_uid_y_x1['label_item_info_submit_click_x1'], df_uid_y_x1['label_item_info_submit_click_x1_pred'])))
print('x2_item：{x2}'.format(x2=roc_auc_score(df_uid_y_x2['label_item_info_submit_click_x2'], df_uid_y_x2['label_item_info_submit_click_x2_pred'])))
print('x3_item：{x3}'.format(x3=roc_auc_score(df_uid_y_x3['label_item_info_submit_click_x3'], df_uid_y_x3['label_item_info_submit_click_x3_pred'])))
print('s：{other}'.format(other=roc_auc_score(df_uid_y_other['label_item_info_submit_click_other'], df_uid_y_other['label_item_info_submit_click_other_pred'])))

In [ ]:
# Recall@30%
print('x1_item：{x1}'.format(x1=cal_recall(df_uid_y_x1, 'label_item_info_submit_click_x1', 'label_item_info_submit_click_x1_pred')))
print('x2_item：{x2}'.format(x2=cal_recall(df_uid_y_x2, 'label_item_info_submit_click_x2', 'label_item_info_submit_click_x2_pred')))
print('x3_item：{x3}'.format(x3=cal_recall(df_uid_y_x3, 'label_item_info_submit_click_x3', 'label_item_info_submit_click_x3_pred')))
print('s：{other}'.format(other=cal_recall(df_uid_y_other, 'label_item_info_submit_click_other', 'label_item_info_submit_click_other_pred')))